In [ ]:
from convertors.convert_to_json import *
from mols_calculation import *
import pandas as pd
import os

In [ ]:
def convert_keys(dict_list):
    """
    Convert keys in metadata_dict based on the provided keys_dict and keys_list.

    :param metadata_dict: A dictionary containing metadata information.
    :return: A dictionary with converted keys based on the provided keys_dict and keys_list.
    """
    output = []
    for metadata_dict in dict_list:
        converted = {keys_dict[key.lower()]: val for key, val in metadata_dict.items() if key.lower() in keys_dict and keys_dict[key.lower()] in keys_list}
    
        converted.update({key: "" for key in keys_list if key not in converted})
        output.append(converted)

    return output

In [ ]:
original_db_path = r"C:\Users\Axel\Documents\DB Janvier 2024\DB_publi\original"

In [ ]:
FINAL_MSP, FINAL_XML, FINAL_CSV, FINAL_JSON, FINAL_MGF = convert_to_json(original_db_path)

In [ ]:
FINAL_MSP = convert_keys(FINAL_MSP)

In [ ]:
FINAL_XML = convert_keys(FINAL_XML)

In [ ]:
FINAL_CSV = convert_keys(FINAL_CSV)

In [ ]:
FINAL_JSON = convert_keys(FINAL_JSON)

In [ ]:
FINAL_MGF = convert_keys(FINAL_MGF)

In [ ]:
msp_df = pd.DataFrame(FINAL_MSP)
del FINAL_MSP

In [ ]:
xml_df = pd.DataFrame(FINAL_XML)
del FINAL_XML

In [ ]:
csv_df = pd.DataFrame(FINAL_CSV)
del FINAL_CSV

In [ ]:
json_df = pd.DataFrame(FINAL_JSON)
del FINAL_JSON

In [ ]:
mgf_df = pd.DataFrame(FINAL_MGF)
del FINAL_MGF

ajouter la colonne 'PREDICTED' et la peupler

passer RDkit sur chacun des spectres

In [ ]:
if not msp_df.empty:
    msp_df = mols_derivation_and_calculation(msp_df)

In [ ]:
if not xml_df.empty:
    xml_df = mols_derivation_and_calculation(xml_df)

In [ ]:
if not csv_df.empty:
    csv_df = mols_derivation_and_calculation(csv_df)

In [ ]:
if not json_df.empty:
    json_df = mols_derivation_and_calculation(json_df)

In [ ]:
if not mgf_df.empty:
    mgf_df = mols_derivation_and_calculation(mgf_df)

écrire les fichiers de sorti

In [ ]:
msp_df.to_csv(r"C:\Users\Axel\PycharmProjects\msp_v3\TOOLS\Generate_csv_from_original_files\output\msp_df.csv", sep=";", quotechar='"', index=False, encoding="UTF-8")

In [ ]:
xml_df.to_csv(r"C:\Users\Axel\PycharmProjects\msp_v3\TOOLS\Generate_csv_from_original_files\output\xml_df.csv", sep=";", quotechar='"', index=False, encoding="UTF-8")

In [ ]:
csv_df.to_csv(r"C:\Users\Axel\PycharmProjects\msp_v3\TOOLS\Generate_csv_from_original_files\output\csv_df.csv", sep=";", quotechar='"', index=False, encoding="UTF-8")

In [ ]:
json_df.to_csv(r"C:\Users\Axel\PycharmProjects\msp_v3\TOOLS\Generate_csv_from_original_files\output\json_df.csv", sep=";", quotechar='"', index=False, encoding="UTF-8")

In [ ]:
mgf_df.to_csv(r"C:\Users\Axel\PycharmProjects\msp_v3\TOOLS\Generate_csv_from_original_files\output\mgf_df.csv", sep=";", quotechar='"', index=False, encoding="UTF-8")